In [54]:
from bs4 import BeautifulSoup, formatter
import pandas as pd
import requests
import re

In [55]:
from datetime import datetime

In [56]:
header = {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/112.0'}
ixigo_ipo = 'https://www.chittorgarh.com/ipo/ixigo-ipo/1742/'

In [57]:
raw_page = requests.get(ixigo_ipo, headers = header).text
soup = BeautifulSoup(raw_page, 'html5')

In [58]:
soup

<!DOCTYPE html>
<html lang="en-US" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>ixigo IPO Date, Price, GMP, Review, Analysis &amp; Details</title>
<meta content="Get ixigo IPO details. Find IPO Date, Price, Live Subscription, Allotment, Grey Market Premium GMP, Listing Date, Analysis and Review." name="description"/>
<link href="https://www.chittorgarh.com/ipo/ixigo-ipo/1742/" rel="canonical"/>
<meta content="https://www.chittorgarh.com/ipo/ixigo-ipo/1742/" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="ixigo IPO Details - IPO Date, Price, GMP, Analysis &amp; Review" property="og:title"/>
<meta content="Get ixigo IPO details. Find IPO Date, Price, Live Subscription, Allotment, Grey Market Premium GMP, Listing Date, Analysis and Review." property="og:description"/>
<meta content="https://www.chittorgarh.com/images/ipo/IXIGO-Logo.png" property="og:image"/>


In [59]:
logo = soup.find_all(class_="div-logo float-start pe-3")[0].find('img')['src']

In [60]:
logo

'https://www.chittorgarh.com/images/ipo/IXIGO-Logo.png'

In [61]:
company_name = soup.find_all(class_="col-md-12")[0].find('h1').text

In [62]:
tables = soup.find_all('table')

In [63]:
ipo_detail_table = {}
for row in tables[1].find_all('tr')[2:]:
    key = row.find_all('td')[0].text
    value = row.find_all('td')[1].text
    ipo_detail_table.update({key:value})
    
ipo_detail_table['Share holding pre issue'] = int(''.join((ipo_detail_table['Share holding pre issue'].split(','))))
ipo_detail_table['Share holding post issue'] = int(''.join((ipo_detail_table['Share holding post issue'].split(','))))
ipo_detail_table['Total Issue Size'] = int(''.join(ipo_detail_table['Total Issue Size'].split(' ')[0].split(',')))
ipo_detail_table['Fresh Issue'] = int(''.join(ipo_detail_table['Fresh Issue'].split(' ')[0].split(',')))
ipo_detail_table['Offer for Sale'] = int(''.join(ipo_detail_table['Offer for Sale'].split(' ')[0].split(',')))
ipo_detail_table['Lot Size'] = int(ipo_detail_table['Lot Size'].split(' ')[0])
ipo_detail_table['Face Value'] = int(re.sub(r"[^\d]","",ipo_detail_table['Face Value'].split(' ')[0]))
ipo_detail_table['Price Band'] = int(re.sub(r"[^\d]","", ipo_detail_table['Price Band'].split(' ')[2]))

In [64]:
ipo_detail_table

{'Face Value': 1,
 'Price Band': 93,
 'Lot Size': 161,
 'Total Issue Size': 79580900,
 'Fresh Issue': 12903226,
 'Offer for Sale': 66677674,
 'Issue Type': 'Book Built Issue IPO',
 'Listing At': 'BSE, NSE',
 'Share holding pre issue': 374519945,
 'Share holding post issue': 387423171}

In [65]:
tables[2].find_all('tr')

[<tr> <th>Investor Category</th> <th>Shares Offered</th><th>Maximum Allottees <i class="fa fa-question-circle ms-2" data-bs-content="Maximum Allottees is the maximum number of RIIs and NIIs to whom shares will be allotted. (applicable in case of oversubscription) &lt;br/&gt;&lt;br/&gt; Maximum Allottees = No. of shares reserved/minimum number of shares to be allotted per respective category.&lt;br/&gt;&lt;br/&gt; Maximum Allottees is NA for other categories, where allotment is on pro-rata basis." data-bs-html="true" data-bs-toggle="popover" title="Maximum Allottees"></i></th> </tr>,
 <tr><td>Anchor Investor Shares Offered</td><td>35,811,405 (45.00%)</td><td>NA</td></tr>,
 <tr><td>QIB Shares Offered</td><td>23,874,271 (30.00%)</td><td>NA</td></tr>,
 <tr><td>NII (HNI) Shares Offered</td><td>11,937,134 (15.00%)</td><td></td></tr>,
 <tr><td>  bNII &gt; ₹10L</td><td>7,958,089 (10.00%)</td><td>3,530</td></tr>,
 <tr><td>  sNII &lt; ₹10L</td><td>3,979,045 (5.00%)</td><td>1,765</td></tr>,
 <tr>

In [66]:
investor_share_holding = {}
for row in tables[2].find_all('tr')[1:]:
    key = re.sub(r"[^a-zA-Z0-9₹>< ]","",row.find_all('td')[0].text)
    value = int(''.join(row.find_all('td')[1].text.split(' ')[0].split(',')))
    investor_share_holding.update({key:value})


In [67]:
investor_share_holding

{'Anchor Investor Shares Offered': 35811405,
 'QIB Shares Offered': 23874271,
 'NII HNI Shares Offered': 11937134,
 'bNII > ₹10L': 7958089,
 'sNII < ₹10L': 3979045,
 'Retail Shares Offered': 7958089,
 'Total Shares Offered': 79580899}

In [86]:
timeline = {}

for row in tables[4].find_all('tr'):
    key = row.find_all('td')[0].text
    try:
        value = datetime.strptime(row.find_all('td')[1].text, "%A, %B %d, %Y").strftime("%d %B %Y")
    except:
        value = datetime.strptime(row.find_all('td')[1].text, "%I %p on %B %d, %Y").strftime("%d %B %Y")
        
    timeline.update({key:value})
    
for row in tables[3].find_all('tr'):
    try:
        key = row.find_all('td')[0].text
        value = datetime.strptime(row.find_all('td')[1].text, "%B %d, %Y").strftime("%d %B %Y")
        timeline.update({key:value})
    except:
        pass
    
sorted_data = (sorted(dict.items(), key=lambda item: datetime.strptime(item[1], '%d %B %Y')))
timeline.clear()
for element in sorted_data:
    timeline.update({element[0]:element[1]})


In [87]:
timeline

{'Bid Date': '07 June 2024',
 'IPO Open Date': '10 June 2024',
 'IPO Close Date': '12 June 2024',
 'Cut-off time for UPI mandate confirmation': '12 June 2024',
 'Basis of Allotment': '13 June 2024',
 'Initiation of Refunds': '14 June 2024',
 'Credit of Shares to Demat': '14 June 2024',
 'Listing Date': '18 June 2024',
 'Anchor lock-in period end date for 50% shares (30 Days)': '13 July 2024',
 'Anchor lock-in period end date for remaining shares (90 Days)': '11 September 2024'}

In [90]:
timeline.

AttributeError: 'dict' object has no attribute 'begin'

In [159]:
//Improve this function

def sortTimeline(my_dict = timeline):
    sorted_dict = {}
    for key, value_list in my_dict.items():
    # Ensure value_list is a list (handles potential single value case)
        if not isinstance(value_list, list):
            value_list = [value_list]
        sorted_dict[key] = sorted(value_list, key=operator.itemgetter(0))  # Sort by first element (datetime)
    return sorted_dict

In [164]:
tables[10]

<table class="table table-sm">
<tbody>
<tr>
<td><b>Listing Date</b></td>
<td>June 18, 2024</td>
</tr>
<tr>
<td><b>BSE Script Code</b></td>
<td>544192</td>
</tr>
<tr>
<td><b>NSE Symbol</b></td>
<td>IXIGO</td>
</tr>
<tr>
<td><b>ISIN</b></td>
<td>INE0HV901016</td>
</tr>
<tr>
<td><b>Final Issue Price</b></td>
<td>₹93 per share</td>
</tr>
</tbody>
</table>

In [16]:
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'

In [12]:
import time
from datetime import datetime

In [77]:
dict = {'IPO Open Date': '10 June 2024',
         'IPO Close Date': '12 June 2024',
         'Basis of Allotment': '13 June 2024',
         'Initiation of Refunds': '14 June 2024',
         'Credit of Shares to Demat': '14 June 2024',
         'Listing Date': '18 June 2024',
         'Cut-off time for UPI mandate confirmation': '12 June 2024',
         'Bid Date': '07 June 2024',
         'Anchor lock-in period end date for 50% shares (30 Days)': '13 July 2024',
         'Anchor lock-in period end date for remaining shares (90 Days)': '11 September 2024'
       }

In [82]:
sorted_data = (sorted(dict.items(), key=lambda item: datetime.strptime(item[1], '%d %B %Y')))

print(sorted_data)

[('Bid Date', '07 June 2024'), ('IPO Open Date', '10 June 2024'), ('IPO Close Date', '12 June 2024'), ('Cut-off time for UPI mandate confirmation', '12 June 2024'), ('Basis of Allotment', '13 June 2024'), ('Initiation of Refunds', '14 June 2024'), ('Credit of Shares to Demat', '14 June 2024'), ('Listing Date', '18 June 2024'), ('Anchor lock-in period end date for 50% shares (30 Days)', '13 July 2024'), ('Anchor lock-in period end date for remaining shares (90 Days)', '11 September 2024')]


In [11]:
(list(dict.values())[0])

AttributeError: 'str' object has no attribute 'astype'

In [27]:
dict['IPO Open Date']

'10 June 2024'

In [18]:
datetime.now()

datetime.datetime(2024, 7, 3, 20, 18, 37, 869744)

In [22]:
datetime.ctime()

TypeError: unbound method datetime.ctime() needs an argument

In [73]:
datetime.strptime('12 June 2025', "%d %B %Y").timestamp()

1749666600.0

In [31]:
dict ={
    'imran':100,
    'Arju': 67,
    'Israt': 89,
    'shazeb': 56
}



In [75]:
sorted(dict)

['Arju', 'Israt', 'imran', 'shazeb']

AttributeError: 'dict_keys' object has no attribute 'any'

In [45]:
for (k,v) in dict.items():

imran 100
Arju 67
Israt 89
shazeb 56


In [46]:
{sorted for (k,v) in dict.items()}

{('Arju', 67), ('Israt', 89), ('imran', 100), ('shazeb', 56)}

In [49]:
sorted_value = sorted(dict.values())

In [ ]:
dict

In [91]:
import flask

ModuleNotFoundError: No module named 'flask'

In [ ]:
new_dict = {}
for i in sorted_value:
    new_dict.update(dict.ke)